In [1]:
# Import appropriate libraries
import pandas
import numpy as np 
import json, requests, urllib, io, math

In [2]:
# Gets the pitch data csv
def get_total_pitch_data():

  pitchData = pandas.read_csv("all_pitch.csv", error_bad_lines=False)
  pitchData = pitchData.rename({"res":"result"}, axis = 1)
  del pitchData["zone"]
  pitchData["zone"] = pitchData['zones']
  pitchData["zones"] = pitchData['zones'].map(lambda x: x.rstrip('aAbB')).astype(int)
  
  return pitchData


total_pitch_data = get_total_pitch_data()



/project/engineering/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
# Gets if an event is an on-base event

def get_ob(row):
    on_base_events = ['Single', 'Walk', 'Double', 'Home Run', 'Hit By Pitch', 'Triple']
    value = row["event"]
    if value in on_base_events:
        return 1
    else:
        return 0

In [3]:



# Gets at-bat data csv
def get_at_bat_data():
    at_bats = pandas.read_csv("all_atbat.csv")
    #get rid of Sac Bunt, Intentional walk, Runner Out, Batter Interference, Catcher Interference because not included when computing OBP
    at_bat_remove_events = ['Sac Bunt', 'Intent Walk', 'Runner Out', 'Batter Interference', 'Catcher Interference']

    at_bats = at_bats.loc[~at_bats['event'].isin(at_bat_remove_events)]
    
    at_bats["on_base"] = at_bats.apply(get_ob, axis=1)
    
    return at_bats

at_bats = get_at_bat_data()

at_bats.head()

,Unnamed: 0,ab_id,batter_id,event,pitcher_id,on_base
0,0,2.019000e+09,594777,Flyout,571666,0
1,1,2.019000e+09,545361,Flyout,571666,0
2,2,2.019000e+09,571506,Groundout,571666,0
3,3,2.019000e+09,543257,Single,502239,1
4,4,2.019000e+09,656305,Flyout,502239,0


In [4]:
# Create list of unique pitchers and batters 

unique_pitchers = at_bats["pitcher_id"].unique()
unique_batters = at_bats["batter_id"].unique()

In [5]:
# Ranks batters and returns a sorted list

def get_ranked_batters(unique_batters, df):
    unranked_batters = []
    for batter in unique_batters:
        shortened_df = df[df["batter_id"]==batter]
        if shortened_df.shape[0] >= 30:
            obp = np.mean(shortened_df["on_base"].to_numpy())
            unranked_batters.append((obp,batter))


    ranked_batters = sorted(unranked_batters, key=lambda x: x[0])
    return ranked_batters
ranked_batters = get_ranked_batters(unique_batters, at_bats)

In [6]:
# Ranks pitchers and returns a sorted list

def get_ranked_pitchers(unique_pitcher, df):
    unranked_pitchers = []
    for pitcher in unique_pitchers:
        shortened_df = df[df["pitcher_id"]==pitcher]
        if shortened_df.shape[0] >= 30:
            obp = np.mean(shortened_df["on_base"].to_numpy())
            unranked_pitchers.append((obp,pitcher))

    ranked_pitchers = sorted(unranked_pitchers, reverse=True, key=lambda x: x[0])
    return ranked_pitchers
ranked_pitchers = get_ranked_pitchers(unique_pitchers, at_bats)

In [8]:
# Returns a dict of 10 lists for players sorted into decile categories (90+ percentile, 80+ percentile . . .)

def get_deciles(ranked_players):
    decile_width = len(ranked_players)/10

    
    decile_map = {}
    
    for i,player in enumerate(ranked_players):
        decile = math.floor(i/decile_width)
        if decile not in decile_map.keys():
            decile_map[decile] = []
        decile_map[decile].append(str(player[1]))

    return decile_map
pitcher_deciles = get_deciles(ranked_pitchers)
batter_deciles = get_deciles(ranked_batters)

decile_dict = {"pitchers":pitcher_deciles,
              "batters":batter_deciles}
with open("deciles.json", "w") as outfile: 
    json.dump(decile_dict, outfile)

In [9]:
# Returns a dict of 4 lists for players sorted into 4 categories ( 75+ percentile, 50+ percentile, . . . )

def get_quartiles(ranked_players):
    quartile_width = len(ranked_players)/4

    
    quartile_map = {}
    
    for i,player in enumerate(ranked_players):
        quartile = math.floor(i/quartile_width)
        if quartile not in quartile_map.keys():
            quartile_map[quartile] = []
        quartile_map[quartile].append(str(player[1]))

    return quartile_map

pitcher_quartiles = get_quartiles(ranked_pitchers)
batter_quartiles = get_quartiles(ranked_batters)

quartile_dict = {"pitchers":pitcher_quartiles,
              "batters":batter_quartiles}
with open("quartiles.json", "w") as outfile: 
    json.dump(quartile_dict, outfile)

In [7]:
# Returns a dict of 3 lists for players sorted into 3 categories (top third, middle third, bottom third)

def get_thirds(ranked_players):
    third_width = len(ranked_players)/3

    
    third_map = {}
    
    for i,player in enumerate(ranked_players):
        third = math.floor(i/third_width)
        third_map[str(player[1])] = third
        

    return third_map

pitcher_thirds = get_thirds(ranked_pitchers)
batter_thirds = get_thirds(ranked_batters)

third_dict = {"pitchers":pitcher_thirds,
              "batters":batter_thirds}
with open("thirds.json", "w") as outfile: 
    json.dump(third_dict, outfile)

In [8]:
# Returns n most seen players in each category 

def get_most_seen(total_pitch_data, thirds, n):
    seen_pitchers, count = np.unique(total_pitch_data["pitcher_id"].to_numpy(),return_counts = True)
    sort_indicies = np.argsort(-count)
    sorted_seen_pitchers = seen_pitchers[sort_indicies]
    
    selected_thirds = {"pitchers":{0:[],1:[],2:[]},
                      "batters":{0:[],1:[],2:[]}}
    for pitcher in sorted_seen_pitchers:
        group = thirds["pitchers"][str(pitcher)]
        if len(selected_thirds["pitchers"][group])<n:
            selected_thirds["pitchers"][group].append(pitcher)
        all_full = True
        for key in selected_thirds["pitchers"].keys():
            if len(selected_thirds["pitchers"][key])<n:
                all_full = False
        if all_full == True:  
            break
    
    seen_batters, count = np.unique(total_pitch_data["batter_id"].to_numpy(),return_counts = True)
    sort_indicies = np.argsort(-count)
    sorted_seen_batters = seen_batters[sort_indicies]
    
    for batter in sorted_seen_batters:
        group = thirds["batters"][str(batter)]
        if len(selected_thirds["batters"][group])<n:
            selected_thirds["batters"][group].append(batter)
        all_full = True
        for key in selected_thirds["batters"].keys():
            if len(selected_thirds["batters"][key])<n:
                all_full = False
        if all_full == True:
            break    
    
    return selected_thirds
        
        
    
    
    

print(get_most_seen(total_pitch_data, third_dict, 5))




{'pitchers': {0: [527048, 451596, 501957, 503449, 543022], 1: [489119, 430935, 502043, 592662, 543699], 2: [453286, 452657, 518516, 519242, 527054]}, 'batters': {0: [572287, 429667, 488721, 595978, 543376], 1: [453943, 448801, 405395, 446334, 520471], 2: [502671, 467793, 458015, 605141, 545361]}}
